In [7]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "pr-flowery-realization-1"


In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "example_data/cv_philippos_barabas.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

1


In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What is the person's name?"})

results

{'input': "What is the person's name?",
 'context': [Document(id='1378dfc7-b905-4db1-9751-1f6bcf6202e1', metadata={'source': 'example_data/cv_philippos_barabas.pdf', 'page': 0}, page_content='B. SC. & M. SC. IN ELECTRICAL ENGINEERING | 10/2013 -\n03/2020EDUCATION & PUBLICATIONS\nCOLAW: Cooperative Location Proof \nArchitecture for VANETs based on\nWitnessing (doi.org)\nBARABAS P., REGNATH E., STEINHORST S. | COINS:\nCONFERENCE ON OMNI-LAYER INTELLIGENT SYSTEMS | 2020LANGUAGES\nEnglish\nGerman\nGreek\nChineseInternet of Things\nRelationship Management\nAnalytical ThinkingTechnical CommunicationCloud ComputingPOC/MVP CreationTechnical Architecture\nPost-Sales Support\nProduct Demonstrations\nAdaptabilityTECHNICAL & SOFT SKILLSEngineer and founder with 5 years of experience in the automotive and technical\nconsulting sector. Proven track record in conceptualizing and delivering technical\nsolutions tailored to clients’ needs, managing client relationships, and providing pre-\nsales techni

In [14]:
results = rag_chain.invoke({"input": "what are Philippos' strongest skills? What do you base that on?"})

results

{'input': "what are Philippos' strongest skills? What do you base that on?",
 'context': [Document(id='5f2757e8-64f8-494a-872b-ea8398d27fe5', metadata={'source': 'example_data/cv_philippos_barabas.pdf', 'page': 0}, page_content='+49 176 57868854\npbarabas@proton.me\nPoetschnerstraße 7\n80634 Munich, GermanyCONTACT\nlinkedin.com/in/philipposbarabasPHILIPPOS\nG. BARABAS\nENGINEER & FOUNDER\nDescription: The zero-waste grocery delivery service Terano (terano.de) with its\nintuitive container deposit system for stables proved that grocery shopping can be\nboth sustainable and convenient. \nKey responsibilities:\nManaging warehouse operations, procurement and last-mile delivery\nResponsible for customer and supplier relationships\nSupported on various other startup activities like creating marketing material,\nshop design, legal matters and data analytics\nAchievements: \nTerano generated five figure revenues and was featured on local newsTERANO | 03/2021 - 10/2022\nCO-FOUNDER, COO\nLothar 